In [1]:
import os
import numpy as np
import trimesh
import torch
import sys

In [2]:
data_dir = '../data/car'
filenames = os.listdir(data_dir)
# filename = filenames[0]

# N = 0
# n_samples = 0   
# for filename in tqdm(filenames):
#     if n_samples >= 1500:
#         break
#     file_path = f'../data/{filename}/models/model_normalized.obj'
#     if not os.path.exists(file_path):
#         continue

#     mesh = trimesh.load(file_path, force='mesh')
#     if not isinstance(mesh, trimesh.Trimesh):
#         continue
    
#     N += mesh.vertices.shape[0]
#     n_samples += 1
#     # mesh.scene().show()

# print(f'Total number of points: {N}')

mesh = trimesh.load(f'{data_dir}/{filenames[0]}/models/model_normalized.obj', force='mesh')
point_cloud = mesh.vertices

In [5]:
sys.path.append('../scripts')
import grid

feature_extractor = grid.FeatureExtractor()

point_cloud_tensor = torch.tensor(point_cloud)[None, :, :].float()
features = feature_extractor(point_cloud_tensor)
output_grid = grid.GridPooling.apply(point_cloud_tensor, features)